## 匯入必要套件

In [ ]:
# For drawing
import matplotlib.pyplot as plt
# For data processing
import numpy as np
import os
# For buildind model
import tensorflow as tf
from tensorflow.keras.applications import MobileNet

## 資料處理

In [ ]:
# 從指定的URL下載並提取檔案
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
# 返回下載的檔案路徑，並將檔案提取到預設的位置
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
# 將最終的資料夾路徑存儲
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [ ]:
# 指定訓練集和驗證集的資料夾路徑
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

# 設定 batch_size 和 img_size
BATCH_SIZE = 32
IMG_SIZE = (224, 224) # MobileNet v1 的輸入大小

#建立訓練集
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, #訓練集路徑
                                                            shuffle=True, #隨機選取
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

#建立驗證集
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir, #驗證集路徑
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

In [ ]:
#把部分驗證集資料，當作測試集
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5) #拿1/5(6筆)的資料當作測試集
validation_dataset = validation_dataset.skip(val_batches // 5)  #拿剩下4/5(26筆)的資料當作驗證集

print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

### 調整執行效能記憶體優化

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE) #記憶體優化
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE) #記憶體優化
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE) #記憶體優化

### 影像縮放

In [ ]:
# 建立資料擴增函式
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'), # 水平翻轉
  tf.keras.layers.RandomRotation(0.2), # 旋轉
])

## 建立模型

In [ ]:
# 建立模型 (不包含全連接層)
# weights='imagenet' 表示使用在 ImageNet 上預訓練的權重初始化模型
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
# 取得資料集中的下一個批次
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

### 不更新權重

In [ ]:
# 模型的權重在訓練過程中將不會被更新
base_model.trainable = False
# 印出模型概述
base_model.summary()

In [ ]:
# 建立一個全域平均池化層，將輸入特徵張量的空間維度進行平均壓縮，保留通道維度
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# 獲得每個通道的平均值
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
# 分類器是二值分類(用BinaryCrossentropy)，所以只有一個輸出
# 將全域平均池化後的特徵向量映射到單一預測值
prediction_layer = tf.keras.layers.Dense(1)
# 將全域平均池化後的特徵向量作為輸入，經過全連接層進行預測操作
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3)) #將輸入影像尺寸resize到160x160x3
x = data_augmentation(inputs) #輸入影像資料擴增
x = base_model(x, training=False) #靜止更改 base_model
x = global_average_layer(x) #5x5 Average Pooling 空間平均
x = tf.keras.layers.Dropout(0.2)(x) # 20% 節點隨機設成0輸出
outputs = prediction_layer(x) #換算成機率值
model = tf.keras.Model(inputs, outputs) #定義新模型的名稱

In [ ]:
#設定訓練超參數
base_learning_rate = 0.0001 #學習率
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), #使用二值交叉熵
              metrics=['accuracy']) #以準確率為指標

model.summary()

In [ ]:
print(len(model.trainable_variables)) # 0:weights, 1:bias
weights_=np.array(model.trainable_variables[0])
print(len(weights_)) # 1280 weights
bias_=np.array(model.trainable_variables[1])
print(len(bias_)) # 1 bias

In [ ]:
initial_epochs = 10 # 設定迭代次數

loss0, accuracy0 = model.evaluate(validation_dataset) # 評估模型的損失和準確率

print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
 #新Model的訓練
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset,
                    verbose=2) #只檢視結果

In [ ]:
#繪製訓練與驗證集的準確度與loss歷史曲線
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### 允許更新權重 (做Fine-tune處理)

In [ ]:
base_model.trainable = True # 允許更新權重

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards 從100層調起
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False #前100層不準調整

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])
model.summary()

In [ ]:
len(model.trainable_variables)

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset,
                         verbose=2) #只顯示結果

In [ ]:
# 增添微調訓練的歷史資料(準確度與loss)
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
# 綠線右方是微調訓練的成果
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

## 只有 train_data 和 test_data 的模型訓練

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# 載入MobileNet模型
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# 凍結模型的權重
base_model.trainable = False

# 建立模型架構
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 編譯模型
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 訓練模型
epochs = 10
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=validation_generator)

# 評估模型
test_loss, test_acc = model.evaluate(validation_generator)
print('Test accuracy:', test_acc)